##最終課題

In [12]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

In [13]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [14]:
#開始URL
base_url = "https://www.musashin-u.ac.jp"
#訪問済みURLを記録
visited_urls = set()
#URLとタイトルを格納する辞書
sitemap = {}

In [15]:
def get_page_title(soup):
    #ページのタイトルを取得
    title_tag = soup.find("title")
    return title_tag.string.strip() if title_tag else "タイトルなし"

In [16]:
def is_same_domain(url):
    #同じドメインか確認
    return urlparse(url).netloc == urlparse(base_url).netloc

In [19]:
def crawl_page(url, depth=0, max_depth=3):
    #深度制限のチェック
    if depth > max_depth or url in visited_urls:
        return
    #訪問済みの場合はスキップ
    if url in visited_urls:
        return
    
    if not is_same_domain(url):
        return
    
    try:
        print(f"訪問中: {url}(深度: {depth})")
        #HTTPリクエストを送信
        res=requests.get(url, headers=header, timeout=10)
        res.encoding = res.apparent_encoding
        #ステータスコードのチェック
        if res.status_code != 200:
            print(f"エラー: ステータスコード{res.status_code}")
            return
        #訪問済みに追加
        visited_urls.add(url)
        #HTMLを解析
        soup = BeautifulSoup(res.text, "html.parser")
        #ページタイトルを取得
        title = get_page_title(soup)
        sitemap[url] = title
        print(f"タイトル: {title}")

        #ページ内のすべての<a>タグを取得
        links = soup.find_all('a', href=True)
        #ページ内のすべてのリンクを取得して処理
        for link in links:
            href = link["href"]
            #PDFやファイルを除く
            if href.endswith((".pdf", ".doc", ".xls", ".zip", ".jpg", ".png")):
                continue
            #相対URLを絶対URLに変換
            full_url = urljoin(url, href)
            #再帰的にクロール
            crawl_page(full_url, depth + 1, max_depth)
        time.sleep(0.5)
    except requests.exceptions.RequestException as e:
        print(f"リクエストエラー: {e}")
    except Exception as e:
        print(f"エラー: {e}")
        print()
#メイン処理
print("=" * 60)
print("武蔵野大学サイトマップ抽出開始")
print("=" * 60)
print()

#クロール開始
crawl_page(base_url)

print()
print("=" * 60)
print("クロール完了")
print("=" * 60)
print(f"収集したページ数: {len(sitemap)}")

#結果を表示
print("[収集したサイトマップ]")
print("*"*60)
for url, title in sitemap.items():
    print(f"URL: {url}")
    print(f"タイトル: {title}")
    print("-"*60)

print("処理終了")

武蔵野大学サイトマップ抽出開始

訪問中: https://www.musashin-u.ac.jp(深度: 0)
リクエストエラー: HTTPSConnectionPool(host='www.musashin-u.ac.jp', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10b9917c0>: Failed to resolve 'www.musashin-u.ac.jp' ([Errno 8] nodename nor servname provided, or not known)"))

クロール完了
収集したページ数: 0
[収集したサイトマップ]
************************************************************
処理終了
